In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from glob import glob
import json
import re
import shutil
import os
from os.path import join, isdir

In [ ]:
from matplotlib import pyplot as plt
import matplotlib_pretty_tex as mpt
import seaborn as sns
from matplotlib.ticker import MultipleLocator

In [ ]:
# Control how ladders are drawn
points = ['A', 'B', 'C', 'D', 'A', 'E', 'F', 'B', 'F', 'G', 'C', 'G', 'H', 'D', 'H', 'E']

In [ ]:
scenarios = ['tip_x=0um_y=-1000um', 'Nominal', 'tip_x=0um_y=+1000um']

In [ ]:
vp_sides = ['VPLeft', 'VPRight']

outdir = f'output/{"_".join(scenarios)}/module_positions/'
if isdir(outdir):
    shutil.rmtree(outdir)
os.makedirs(outdir)

velo_geometry = {'local': {}, 'global': {}}
for scenario in scenarios:
    with open(f'output/scenarios/{scenario}/local_geo.json', 'rt') as f:
        velo_geometry['local'][scenario] = json.load(f)
    with open(f'output/scenarios/{scenario}/global_geo.json', 'rt') as f:
        velo_geometry['global'][scenario] = json.load(f)

## Look at the $xy$ plane for all modules

To check that the all line up

In [ ]:
for scenario in velo_geometry['global']:
    plt.figure(figsize=[8, 8], dpi=100)
    for m_id in range(0, 52, 2):
        assert m_id % 2 == 0
        # Draw all ladders for this module
        for vp_side in vp_sides:
            for l_id in range(4):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][0] for p in points]
                ys = [ladder[p][1] for p in points]
                plt.plot(xs, ys, label=f'M{m_id:002d} L{l_id}', lw=1, alpha=0.75)
            # Increment the module id
            m_id += 1

    plt.gca().set_aspect('equal')
    plt.xlabel('Global LHCb X axis [mm]')
    plt.ylabel('Global LHCb Y axis [mm]')
    plt.axvline(0, ls='--', c='k', lw=1)
    plt.axhline(0, ls='--', c='k', lw=1)
    mpt.savefig(join(outdir, f'x_vs_y_all_{scenario}.pgf.png'))

## Look at the $xz$ plane

In [ ]:
plt.figure(figsize=[16, 8], dpi=200)
for scenario, c in zip(velo_geometry['global'], sns.color_palette('colorblind', len(velo_geometry['global']))):
    for m_id in range(0, 52, 2):
        for vp_side in vp_sides:
            mean_z_pos = 0

            # Draw all ladders for this module
            for l_id in range(4):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][2] for p in points]
                ys = [ladder[p][0] for p in points]
                mean_z_pos += sum(xs)/len(xs)/4
                label = [None, scenario][m_id == 0 and l_id == 0]
                plt.plot(xs, ys, label=label, alpha=0.75, c=c, lw=0.25)

            # Draw the position label
            if scenario == 'nominal':
                plt.text(mean_z_pos, 85, f'M{m_id:002d} {mean_z_pos:.2f}mm', fontsize=4, rotation=45)
                # print(f'{m_id}: {mean_z_pos},', end=' ')

            # Increment the module id
            m_id += 1

plt.gca().set_aspect('equal')
plt.legend(loc='best', frameon=True, fontsize=4)
plt.xlabel('Global LHCb Z axis [mm]')
plt.ylabel('Global LHCb X axis [mm]')

mpt.savefig(join(outdir, f'x_vs_z_all.pgf.png'))

## Look at the $yz$ plane

In [ ]:
plt.figure(figsize=[16, 8], dpi=200)
for scenario, c in zip(velo_geometry['global'], sns.color_palette('colorblind', len(velo_geometry['global']))):
    for m_id in range(0, 52, 2):
        for vp_side in vp_sides:
            mean_z_pos = 0

            # Draw all ladders for this module
            for l_id in range(4):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][2] for p in points]
                ys = [ladder[p][1] for p in points]
                mean_z_pos += sum(xs)/len(xs)/4
                label = [None, scenario][m_id == 0 and l_id == 0]
                plt.plot(xs, ys, label=label, alpha=0.75, c=c, lw=0.25)

            # Draw the position label
            if scenario == 'nominal':
                plt.text(mean_z_pos, 85, f'M{m_id:002d} {mean_z_pos:.2f}mm', fontsize=4, rotation=45)

            # Increment the module id
            m_id += 1

plt.gca().set_aspect('equal')
plt.legend(loc='best', frameon=True, fontsize=4)
plt.xlabel('Global LHCb Z axis [mm]')
plt.ylabel('Global LHCb Y axis [mm]')
mpt.savefig(join(outdir, f'y_vs_z_all.pgf.png'))

## Look at the $xy$ plane of a single module pair

In [ ]:
for scenario in velo_geometry['global']:
    plt.figure(figsize=[8, 8], dpi=100)
    for m_id in range(0, 1, 2):
        assert m_id % 2 == 0
        # Draw all ladders for this module
        for vp_side, ls in zip(vp_sides, ['-', '--']):
            for l_id, c in zip(range(4), sns.color_palette('colorblind', 4)):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][0] for p in points]
                ys = [ladder[p][1] for p in points]
                lines = plt.plot(xs, ys, label=f'M{m_id:002d} L{l_id}', lw=1, alpha=0.75, ls=ls, c=c)
                plt.text(
                    ladder['A'][0], ladder['A'][1],
                    f'     ({ladder["A"][0]:.2f}, {ladder["A"][1]:.2f}, {ladder["A"][2]:.2f})     ',
                    fontsize=6, color=c
                )
                plt.text(
                    ladder['B'][0], ladder['B'][1],
                    f'     ({ladder["B"][0]:.2f}, {ladder["B"][1]:.2f}, {ladder["B"][2]:.2f})     ',
                    fontsize=6, color=c, horizontalalignment='right'
                )
                plt.text(
                    ladder['C'][0], ladder['C'][1],
                    f'     ({ladder["C"][0]:.2f}, {ladder["C"][1]:.2f}, {ladder["C"][2]:.2f})     ',
                    fontsize=6, color=c, horizontalalignment='right'
                )
                plt.text(
                    ladder['D'][0], ladder['D'][1],
                    f'     ({ladder["D"][0]:.2f}, {ladder["D"][1]:.2f}, {ladder["D"][2]:.2f})     ',
                    fontsize=6, color=c
                )
            # Increment the module id
            m_id += 1

    plt.gca().set_aspect('equal')
    plt.xlabel('Global LHCb X axis [mm]')
    plt.ylabel('Global LHCb Y axis [mm]')
    plt.axvline(0, ls='--', c='k', lw=1)
    plt.axhline(0, ls='--', c='k', lw=1)

    plt.legend(loc='best', frameon=True)
    plt.title(scenario)
    mpt.savefig(join(outdir, f'x_vs_y_M00_M01_{scenario}.pgf.png'))

## Look at the $yz$ plane of a single module pair

In [ ]:
for scenario in velo_geometry['global']:
    plt.figure(figsize=[8, 8], dpi=100)
    for m_id in range(0, 1, 2):
        assert m_id % 2 == 0
        # Draw all ladders for this module
        for vp_side, ls in zip(vp_sides, ['-', '--']):
            for l_id, c in zip(range(4), sns.color_palette('colorblind', 4)):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][2] for p in points]
                ys = [ladder[p][0] for p in points]
                lines = plt.plot(xs, ys, label=f'M{m_id:002d} L{l_id}', lw=1, alpha=0.75, ls=ls, c=c)
                plt.text(
                    ladder['A'][2], ladder['A'][0],
                    f'     ({ladder["A"][0]:.2f}, {ladder["A"][1]:.2f}, {ladder["A"][2]:.2f})     ',
                    fontsize=6, color=c
                )
                plt.text(
                    ladder['B'][2], ladder['B'][0],
                    f'     ({ladder["B"][0]:.2f}, {ladder["B"][1]:.2f}, {ladder["B"][2]:.2f})     ',
                    fontsize=6, color=c, horizontalalignment='right'
                )
                plt.text(
                    ladder['C'][2], ladder['C'][0],
                    f'     ({ladder["C"][0]:.2f}, {ladder["C"][1]:.2f}, {ladder["C"][2]:.2f})     ',
                    fontsize=6, color=c, horizontalalignment='right'
                )
                plt.text(
                    ladder['D'][2], ladder['D'][0],
                    f'     ({ladder["D"][0]:.2f}, {ladder["D"][1]:.2f}, {ladder["D"][2]:.2f})     ',
                    fontsize=6, color=c
                )
            # Increment the module id
            m_id += 1

    # plt.gca().set_aspect('equal')
    plt.xlabel('Global LHCb Z axis [mm]')
    plt.ylabel('Global LHCb X axis [mm]')

    plt.legend(loc='best', frameon=True)
    plt.title(scenario)
    mpt.savefig(join(outdir, f'x_vs_z_M00_M01_{scenario}.pgf.png'))

In [ ]:
plt.figure(figsize=[12, 8], dpi=100)
for scenario, c in zip(velo_geometry['global'], sns.color_palette('dark', len(velo_geometry['global']))):
    for m_id in range(0, 1, 2):
        assert m_id % 2 == 0
        # Draw all ladders for this module
        for vp_side in vp_sides:
            for l_id in range(4):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][2] for p in points]
                ys = [ladder[p][0] for p in points]
                lines = plt.plot(xs, ys, label=[None, scenario][l_id == m_id == 0], lw=0.25, alpha=0.75, c=c)
            # Increment the module id
            m_id += 1

# plt.gca().set_aspect('equal')
plt.xlabel('Global LHCb Z axis [mm]')
plt.ylabel('Global LHCb X axis [mm]')
plt.gca().xaxis.set_minor_locator(MultipleLocator(1))
plt.grid(b=True, which='minor', color='w', linestyle='--')

plt.legend(loc='best', frameon=True)
mpt.savefig(join(outdir, f'x_vs_z_M00_M01.pgf.png'))

In [ ]:
plt.figure(figsize=[12, 8], dpi=100)
for scenario, c in zip(velo_geometry['global'], sns.color_palette('dark', len(velo_geometry['global']))):
    for m_id in range(0, 1, 2):
        assert m_id % 2 == 0
        # Draw all ladders for this module
        for vp_side in vp_sides:
            for l_id in range(4):
                ladder = velo_geometry['global'][scenario][
                    f'/dd/Structure/LHCb/BeforeMagnetRegion/VP/{vp_side}/'
                    f'Module{m_id:002d}WithSupport/Module{m_id:002d}/Ladder{l_id}'
                ]
                xs = [ladder[p][2] for p in points]
                ys = [ladder[p][0] for p in points]
                lines = plt.plot(xs, ys, label=[None, scenario][l_id == m_id == 0], lw=0.25, alpha=0.75, c=c)
            # Increment the module id
            m_id += 1

# plt.gca().set_aspect('equal')
plt.xlabel('Global LHCb Z axis [mm]')
plt.ylabel('Global LHCb X axis [mm]')
plt.gca().xaxis.set_minor_locator(MultipleLocator(0.1))
plt.grid(b=True, which='minor', color='w', linestyle='--')

plt.legend(loc='best', frameon=True)
mpt.savefig(join(outdir, f'y_vs_z_M00_M01.pgf.png'))